In [ ]:
from langchain_openai import ChatOpenAI
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

Modules for the agent:
- Main Agent Responsible for coordindation
- Parental permission and Managegement
- SCreenshot and Image parsing
- Content contatenation and summarization
- Appeals Agent
- Time Management Agent


In [1]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

# result =  Runner.run_sync(agent, "Write a haiku about recursion in programming.")
result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)


Call itself again,  
A spiral with no ending—  
Base case brings the peace.


In [4]:
from winotify import Notification, audio

toast = Notification(
    app_id="My Python App",
    title="Hello!",
    msg="This is your notification message",
    duration="short"  # short or long
)

toast.show()

In [60]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
    ],
)

    


In [3]:
result = await Runner.run(agent, "What is this website about? https://www.google.com/search?sca_esv=ff5a3d640eaa04bd&sxsrf=AE3TifNO1VemM4bPRUtlys6C2S29VpuKrQ:1760162872037&udm=2&fbs=AIIjpHxU7SXXniUZfeShr2fp4giZ1Y6MJ25_tmWITc7uy4KIeuYzzFkfneXafNx6OMdA4MQRJc_t_TQjwHYrzlkIauOKj9nSuujpEIbB1x32lFLEvHqOmFe_-41rWC15qkwKqyh1Adl28caI-H4UkF8vEkZPiKeLm1-IF950TIh8vlACO2mbTPf8VBxPKSqKtr1s8uQjQLdEjaVIu_0_-ZWXvwTw3fTWnw&q=Harambe&sa=X&ved=2ahUKEwjD_-_VvZuQAxXmiO4BHaRCHdcQtKgLegQIERAB&biw=1536&bih=742&dpr=1.25")
print(result.final_output)

You’ve shared a Google Search results URL for the query "Harambe." Here's what the page is generally about:

• This is not a typical website with content like an article or blog—it's a Google Search results page. Google Search aggregates and displays relevant information for whatever you search, including links to web pages, images, news articles, videos, and more.

• In this case, you’ve searched for the term "Harambe," which triggers Google to show a variety of related content—such as images, news, videos, and related searches—all associated with Harambe.

• Essentially, this page is a gateway to various resources across the web that mention or discuss "Harambe."


In [61]:
web_analysis_prompt = """
You are an expert content analyst specialising in evaluating web content for child safety and educational value.
The parent has requested the following standards for content evaluation: {parental_prompt}.
Your task is to analyze the provided content and determine if it aligns with these standards:
URL: {url}
Title: {title}
Content: {content}

if the content is lacking or insufficeint to make a decision, you can call the WebSearchTool to gather more information about the webpage. 
After which you can take the call whether to approve or block the website. The output has to be in the following JSON format:
{{
    "link" : "<if the content is bad but the website is good, provide the link to the content only, else if the website is bad then add the domain link. Eg: www.example.com or www.example.com/badcontent.html>",
    "decision": "<'approve', 'block'>",
    "reason": "<A brief reason for your decision, maximum 50 words. Be specific about what content or aspect led to your decision.>",
    }}
"""

In [44]:
web_analysis_prompt = """
You are an expert content analyst specialising in evaluating web content for child safety and educational value.
The parent has requested the following standards for content evaluation: {parental_prompt}.
Your task is to analyze the provided content and determine if it aligns with these standards:
URL: {url}
Title: {title}
Content: {content}
Your first task is to use the WebSearchTool to gather more information about the webpage and after which you should evaluate the content.
After which you can take the call whether to approve or block the website. The output has to be in the following JSON format:
{{
    "link" : "<if the content is bad but the website is good, provide the link to the content only, else if the website is bad then add the domain link. Eg: www.example.com or www.example.com/badcontent.html>",
    "decision": "<'approve', 'block'>",
    "reason": "<A brief reason for your decision, maximum 50 words. Be specific about what content or aspect led to your decision.>"
}}
"""


In [62]:
from pydantic import BaseModel
class web_content_analysis_JSON(BaseModel):
    link: str
    decision: str  # 'approve' or 'block'
    reason: str

web_checker_agent = Agent(
    name="Web Content Checker Agent",
    tools=[
        WebSearchTool(),
    ],
    output_type = web_content_analysis_JSON
)

In [45]:
web_analysis_prompt.format(
    parental_prompt="No violent content, no adult content, educational value only",
    url=r"www.pornhub.com",
    title="Gandhi fights back against the British rule",
    content=""" 
Mahatma Gandhi emerged as the foremost leader of India’s freedom struggle against British colonial rule. Rejecting violence, he pioneered the philosophy of Satyagraha (truth-force) and Ahimsa (non-violence) as tools of resistance. Through movements like the Non-Cooperation Movement (1920–22), the Salt March (1930), and the Quit India Movement (1942), Gandhi mobilized millions of Indians across classes and communities to peacefully defy unjust British laws and institutions.
Gandhi’s approach challenged the might of the British Empire not through armed conflict but through mass civil disobedience, boycotts of British goods, and constructive programs such as promoting khadi (hand-spun cloth) and rural self-reliance. His methods exposed the moral contradictions of colonialism and garnered global attention, ultimately accelerating India’s path to independence in 1947.
Even when faced with imprisonment, repression, and violence, Gandhi’s unwavering commitment to non-violence inspired a generation of freedom fighters and set a lasting precedent for civil rights struggles worldwide.
"""
)

'\nYou are an expert content analyst specialising in evaluating web content for child safety and educational value.\nThe parent has requested the following standards for content evaluation: No violent content, no adult content, educational value only.\nYour task is to analyze the provided content and determine if it aligns with these standards:\nURL: www.pornhub.com\nTitle: Gandhi fights back against the British rule\nContent:  \nMahatma Gandhi emerged as the foremost leader of India’s freedom struggle against British colonial rule. Rejecting violence, he pioneered the philosophy of Satyagraha (truth-force) and Ahimsa (non-violence) as tools of resistance. Through movements like the Non-Cooperation Movement (1920–22), the Salt March (1930), and the Quit India Movement (1942), Gandhi mobilized millions of Indians across classes and communities to peacefully defy unjust British laws and institutions.\nGandhi’s approach challenged the might of the British Empire not through armed conflict

In [50]:
result = await Runner.run(web_checker_agent, web_analysis_prompt.format(
    parental_prompt="No violent content, no adult content, educational value only",
    url=r"https://www.youtube.com/watch?v=WB_XA7bINKE&t=81s",
    title="Gandhi fights back against the British rule",
    content=""" 
Trenging topics. Login. Sign up. Home. Videos. Categories. Channels.  Community. Store. Premium. More. Search. Search for videos, photos, models, users, groups, channels, etc.
"""
))
print(result.final_output)

link='www.youtube.com/watch?v=WB_XA7bINKE' decision='approve' reason="The video appears to be educational about Gandhi's non‑violent resistance to British rule. There’s no indication of violence or adult content, and it provides clear historical and educational value."


In [51]:
print(result.final_output.model_dump_json(indent=4))

{
    "link": "www.youtube.com/watch?v=WB_XA7bINKE",
    "decision": "approve",
    "reason": "The video appears to be educational about Gandhi's non‑violent resistance to British rule. There’s no indication of violence or adult content, and it provides clear historical and educational value."
}


In [53]:
from youtube_transcript_api import YouTubeTranscriptApi

ytt_api = YouTubeTranscriptApi()
ytt_api.fetch("6Lq3k-XQkrE")

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Hey, first of all, can we all agree that', start=0.0, duration=3.76), FetchedTranscriptSnippet(text='going to Pizza Hut in the early 2000s', start=1.68, duration=4.72), FetchedTranscriptSnippet(text='was so [\xa0__\xa0] hot', start=3.76, duration=4.96), FetchedTranscriptSnippet(text='>> on', start=6.4, duration=4.159), FetchedTranscriptSnippet(text='when we were around and maybe even', start=8.72, duration=3.919), FetchedTranscriptSnippet(text='before our time slightly was like the', start=10.559, duration=4.24), FetchedTranscriptSnippet(text='hypest and coolest part of being a kid.', start=12.639, duration=3.2), FetchedTranscriptSnippet(text='Like we talked about this before, but', start=14.799, duration=2.081), FetchedTranscriptSnippet(text="like going to McDonald's.", start=15.839, duration=2.641), FetchedTranscriptSnippet(text='>> Yeah, I was going to say yo,', start=16.88, duration=4.479), FetchedTranscriptSnippet(text="co

In [67]:
from urllib.parse import urlparse, parse_qs
def extract_video_id(url: str) -> str:
    """
    Extract video ID from various YouTube URL formats.
    Supports: youtube.com/watch?v=ID, youtu.be/ID, youtube.com/embed/ID
    """
    # Pattern for youtu.be short links
    if 'youtu.be/' in url:
        return url.split('youtu.be/')[-1].split('?')[0].split('&')[0]
    
    # Pattern for youtube.com links
    parsed_url = urlparse(url)
    if parsed_url.hostname in ['www.youtube.com', 'youtube.com', 'm.youtube.com']:
        if parsed_url.path == '/watch':
            query_params = parse_qs(parsed_url.query)
            return query_params.get('v', [None])[0]
        elif parsed_url.path.startswith('/embed/'):
            return parsed_url.path.split('/embed/')[-1].split('?')[0]
        elif parsed_url.path.startswith('/v/'):
            return parsed_url.path.split('/v/')[-1].split('?')[0]
    
    return None

In [68]:
extract_video_id("https://www.youtube.com/watch?v=6Lq3k-XQkrE")

'6Lq3k-XQkrE'

In [79]:
from agents import function_tool, Agent, Runner
@function_tool
def get_youtube_transcript(url: str) -> dict:
    """
    Extracts the transcript from a YouTube video URL.
    
    Args:
        url: The YouTube video URL (supports youtube.com and youtu.be formats)
        
    Returns:
        dict: Contains 'success' (bool), 'transcript' (str), and 'error' (str) if failed
    """
    try:
        # Extract video ID from URL
        video_id = extract_video_id(url)
        if not video_id:
            return {
                "success": False,
                "transcript": "",
                "error": "Invalid YouTube URL"
            }
        
        # Get transcript using the new API (v1.2.2+)
        ytt_api = YouTubeTranscriptApi()
        fetched_transcript = ytt_api.fetch(video_id)
        
        # Combine all transcript snippets into single text
        full_transcript = " ".join([snippet.text for snippet in fetched_transcript.snippets])
        
        return {
            "success": True,
            "transcript": full_transcript,
            "error": ""
        }
        
    except Exception as e:
        return {
            "success": False,
            "transcript": "",
            "error": f"Error extracting transcript: {str(e)}"
        }



def extract_video_id(url: str) -> str:
    if 'youtu.be/' in url:
        return url.split('youtu.be/')[-1].split('?')[0].split('&')[0]
    
    # Pattern for youtube.com links
    parsed_url = urlparse(url)
    if parsed_url.hostname in ['www.youtube.com', 'youtube.com', 'm.youtube.com']:
        if parsed_url.path == '/watch':
            query_params = parse_qs(parsed_url.query)
            return query_params.get('v', [None])[0]
        elif parsed_url.path.startswith('/embed/'):
            return parsed_url.path.split('/embed/')[-1].split('?')[0]
        elif parsed_url.path.startswith('/v/'):
            return parsed_url.path.split('/v/')[-1].split('?')[0]
    
    return None

agent = Agent(
    name="Parental Web Content Analyzer",
    instructions="""You are an expert content analyst specializing in evaluating web content for child safety and educational value.
    
When you receive a YouTube video URL:
1. Use the get_youtube_transcript tool to extract the video transcript
2. Analyze the transcript content based on the parental standards provided
3. Make your decision to approve or block based on the transcript content

For other web content:
1. Analyze the provided title and content
2. If content is insufficient, you can use web search to gather more information
3. Make your decision based on all available information

Always return your decision in the required JSON format.""",
    tools=[
        WebSearchTool(), get_youtube_transcript
    ],
)

In [82]:
result = await Runner.run(agent, web_analysis_prompt.format(
    parental_prompt="No violent content, no adult content, educational value only",
    url=r"https://www.youtube.com/watch?v=Ai8e383CEcc",
    title="Farmer grows 1 billion sunflowers",
    content=""" 
Trenging topics. Login. Sign up. Home. Videos. Categories. Channels.  Community. Store. Premium. More. Search. Search for videos, photos, models, users, groups, channels, etc.
"""
))

In [83]:
print(result.final_output)

{
    "link": "https://www.youtube.com/watch?v=Ai8e383CEcc",
    "decision": "block",
    "reason": "Transcript contains profanity, references to drugs and violence (e.g., discussion of meth labs, gun parts, harming animals, and explicit plans for violent acts). Not appropriate for children and fails educational-only standard."
}


In [77]:
def get_youtube_transcript_test(url: str) -> dict:
    """
    Extracts the transcript from a YouTube video URL.
    
    Args:
        url: The YouTube video URL (supports youtube.com and youtu.be formats)
        
    Returns:
        dict: Contains 'success' (bool), 'transcript' (str), and 'error' (str) if failed
    """
    try:
        # Extract video ID from URL
        video_id = extract_video_id(url)
        if not video_id:
            return {
                "success": False,
                "transcript": "",
                "error": "Invalid YouTube URL"
            }
        
        # Get transcript using the new API (v1.2.2+)
        ytt_api = YouTubeTranscriptApi()
        fetched_transcript = ytt_api.fetch(video_id)
        
        # Combine all transcript snippets into single text
        full_transcript = " ".join([snippet.text for snippet in fetched_transcript.snippets])
        
        return {
            "success": True,
            "transcript": full_transcript,
            "error": ""
        }
        
    except Exception as e:
        return {
            "success": False,
            "transcript": "",
            "error": f"Error extracting transcript: {str(e)}"
        }

In [78]:
get_youtube_transcript_test("https://www.youtube.com/watch?v=6Lq3k-XQkrE")

{'success': True,
 'transcript': 'Hey, first of all, can we all agree that going to Pizza Hut in the early 2000s was so [\xa0__\xa0] hot >> on when we were around and maybe even before our time slightly was like the hypest and coolest part of being a kid. Like we talked about this before, but like going to McDonald\'s. >> Yeah, I was going to say yo, corporations, let\'s go, dude. It was sick, dude. It was sick. It was sick, man. Gamers, welcome back to another episode of Trash Taste. I\'m your host, Connor. Once again, joined by the boys and God. So, >> uh, yeah. >> Look, the one thing people always yap about online is things were better when I was a kid. So, we\'re going to put it to the test today. >> We\'re going to be doing >> 2005 versus 2025. We\'re going to share a bunch of different things and we just decide which one\'s better. Is end shitification real? Does that get what get you demonetized? >> Shitification. >> End shitification. >> It end. >> Well, you said the word. I\'v